In [7]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## ChatOpenAI

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://platform.openai.com/docs/models


In [4]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
print(f"[답변]: {llm.invoke(question).content}")

[답변]: 대한민국의 수도는 서울이야.


In [9]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

chatgpt = ChatOpenAI()
anthropic = ChatAnthropic(model="claude-3.5")

## 프롬프트 템플릿의 활용


`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.


In [10]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체


`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다
- 사용법
  - `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
  - `llm`: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.


In [11]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

c:\Users\donchang\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### run()


`run()` 함수로 템플릿 프롬프트 실행


In [15]:
# 체인 실행: run()
llm_chain.run(country="일본")

'일본의 수도는 도쿄입니다.'

In [16]:
# 체인 실행: run()
llm_chain.run(country="캐나다")

'캐나다의 수도는 오타와(Ottawa)입니다.'

`predict()` 함수로 `run()` 함수를 대체할 수 있습니다.


In [17]:
llm_chain.predict(country="대한민국")

'대한민국의 수도는 서울이야.'

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [18]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

llm_chain.apply(input_list)

[{'text': '호주의 수도는 캔버라입니다.'},
 {'text': '중국의 수도는 베이징(北京)입니다.'},
 {'text': '네덜란드의 수도는 암스테르담입니다.'}]

`text` 키 값으로 결과 뭉치가 반환되었음을 확인할 수 있습니다.

이를 반복문으로 출력한다면 다음과 같습니다.


In [19]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


### `generate()`


`generate()` 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.


In [20]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cc101020-70c4-46b3-b094-050b50db2c00-0', usage_metadata={'input_tokens': 17, 'output_tokens': 13, 'total_tokens': 30}))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-95292d3b-1c12-43f9-944d-16044b610654-0', usage_metadata={'input_tokens': 19, 'output_tokens': 18, 'total_tokens': 37}))], [ChatGeneration(t

In [23]:
# 답변 출력
generated_result.generations

[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cc101020-70c4-46b3-b094-050b50db2c00-0', usage_metadata={'input_tokens': 17, 'output_tokens': 13, 'total_tokens': 30}))],
 [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-95292d3b-1c12-43f9-944d-16044b610654-0', usage_metadata={'input_tokens': 19, 'output_tokens': 18, 'total_tokens': 37}))],
 [ChatGeneration(text='네덜란드의

In [24]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo'}

In [25]:
# run ID 출력
generated_result.run

[RunInfo(run_id=UUID('cc101020-70c4-46b3-b094-050b50db2c00')),
 RunInfo(run_id=UUID('95292d3b-1c12-43f9-944d-16044b610654')),
 RunInfo(run_id=UUID('0a892ce8-b4e6-4b9f-9625-94d2596848dc'))]

In [26]:
# 답변 출력
for gen in generated_result.generations:
    print(gen[0].text.strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


### 2개 이상의 변수를 템플릿 안에 정의


2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(`input_variables`) 를 활용하여 템플릿 구성을 해보겠습니다.


In [27]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [28]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [29]:
# 체인 실행: run()
print(llm_chain.run(area1="서울", area2="파리"))

서울과 파리의 시차는 8시간입니다. 서울은 GMT+9 시간대에 속하고, 파리는 GMT+1 시간대에 속하기 때문에 시차가 8시간이 발생합니다.


In [30]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 표준시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST 또는 GMT-5)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 19시간입니다. 서울은 GMT+9 시간대에 속하고, 하와이는 GMT-10 시간대에 속하기 때문에 시차가 19시간이 발생합니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 GMT+10 시간대에 속하고, 베이징은 GMT+8 시간대에 속하기 때문입니다.


## 스트리밍(streaming)

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 `streaming=True` 로 설정하고 스트리밍으로 답변을 받기 위한 `StreamingStdOutCallbackHandler()` 을 콜백으로 지정합니다.


In [31]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [32]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.predict(question)

대한민국은 동아시아에 위치한 고도로 발전한 선진국가로, 수도는 서울이며 인구는 약 5천만 명이 넘는 대국이다. 대한민국은 민주공화국으로 정부는 대통령 중심의 대통령제를 채택하고 있다. 국회는 한국 국회로서 의회의 기능을 수행하며, 법원은 대한민국 헌법재판소와 대법원으로 구성되어 있다.

대한민국은 세계에서 가장 혁신적이고 발전된 IT 기술을 보유한 나라 중 하나로, 삼성, LG, 현대 등 세계적인 기업들이 본사를 두고 있다. 또한 K-pop, K-drama 등 한류 문화 열풍이 전 세계적으로 인기를 끌고 있으며, 한식, 한복, 태권도 등 전통문화도 세계적으로 알려져 있다.

대한민국은 경제적으로도 세계에서 주요한 역할을 하고 있으며, 선진국가로서 교육, 의료, 인프라 등 다양한 분야에서 높은 수준의 발전을 이루어내고 있다. 또한 한반도에 위치하여 북한과의 관계가 중요한 이슈로 대두되고 있으며, 국제사회에서도 중요한 역할을 하고 있다. 대한민국은 다양한 면에서 매력적인 나라로, 많은 이들이 그 문화와 경제력을 경험하기 위해 방문하고 있다.